In [19]:
import json
from pathlib import Path
import pandas as pd

In [20]:
questions_path = "/kaggle/input/evaluation/questions_template.jsonl"

with open(questions_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, 1):
        print(i, line.strip())

1 {"id":"q1","question":"آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند متر اعلام شد؟","reference_answer":"بین ۳۰ تا ۴۰ متر","doc_ids":[205120]}
2 {"id":"q4","question":"آزمون مقیاس‌های تکاملی شیرخواران (Bayley) عملکرد تکاملی کودکان چه بازه سنی را بررسی می‌کند؟","reference_answer":"کودکان ۱ تا ۴۲ ماهه","doc_ids":[300712]}
3 {"id":"q6","question":"طبق گفته مهرداد بذرپاش، همه بانک‌ها باید چه درصدی از تسهیلات خود را به بخش مسکن تخصیص دهند؟","reference_answer":"۲۰ درصد","doc_ids":[230376]}
4 {"id":"q7","question":"طبق گفته علیرضا پاکدل، حریفان مستقیم ایران برای کسب سهمیه جهانی هندبال بانوان در قهرمانی آسیا کدام تیم‌ها هستند؟","reference_answer":"هنگ‌کنگ، تایلند و ازبکستان","doc_ids":[96027]}
5 {"id":"q8","question":"در خبر دیدار وزیر خارجه ونزوئلا با امیرعبداللهیان، گفته شد در سفر آتی نیکلاس مادورو به تهران چه سندی امضا می‌شود؟","reference_answer":"سند همکاری ۲۰ ساله دو کشور","doc_ids":[150137]}
6 {"id":"q11","question":"طبق گفته مدیرکل سیاسی استانداری سمنان، انتخابات ۱۴۰۰ در سمنان به چه شی

In [21]:
def read_questions_jsonl(path):
    questions = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            questions.append({
                "id": obj["id"],
                "question": obj["question"],
                "reference_answer": obj["reference_answer"],
                "doc_ids": obj["doc_ids"],
            })
    return questions

questions = read_questions_jsonl(questions_path)
print("num_questions:", len(questions))
print(questions[0])

num_questions: 15
{'id': 'q1', 'question': 'آبخور اسکله\u200cهای «بندر بزرگ خلیج فارس» بین چند متر اعلام شد؟', 'reference_answer': 'بین ۳۰ تا ۴۰ متر', 'doc_ids': [205120]}


In [22]:
modes = ["semantic", "lexical", "hybrid"]
results = []

for q in questions:
    for mode in modes:
        retrieved_df = retrieve(q["question"], top_k=5, embedding_model=mode)

        if "rag_answer" in globals():
            ans_text, _ = rag_answer(q["question"], top_k=5, embedding_model=mode)
        else:
            ans_text = answer(q["question"], top_k=5, embedding_model=mode)

        results.append({
            "id": q["id"],
            "embedding_model": mode,
            "question": q["question"],
            "reference_answer": q["reference_answer"],
            "doc_ids": q["doc_ids"],
            "retrieved_doc_ids": retrieved_df["doc_id"].tolist(),
            "retrieved_chunk_ids": retrieved_df["chunk_id"].tolist(),
            "retrieved_scores": retrieved_df["score"].tolist() if "score" in retrieved_df.columns else [],
            "answer": ans_text,
        })

results_df = pd.DataFrame(results)
results_df.head()

,id,embedding_model,question,reference_answer,doc_ids,retrieved_doc_ids,retrieved_chunk_ids,retrieved_scores,answer
0,q1,semantic,آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند ...,بین ۳۰ تا ۴۰ متر,[205120],"[66704, 193981, 92336, 324524, 235045]","[44967, 5016, 38742, 17324, 33708]","[0.7057710886001587, 0.7026494741439819, 0.696...",اطلاعات کافی در متن‌های بازیابی‌شده نیست.
1,q1,lexical,آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند ...,بین ۳۰ تا ۴۰ متر,[205120],"[260634, 235045, 197211, 235045, 235045]","[69722, 33709, 63846, 33706, 33710]","[0.7080084681510925, 0.5838674306869507, 0.553...",اطلاعات کافی در متن‌های بازیابی‌شده نیست.
2,q1,hybrid,آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند ...,بین ۳۰ تا ۴۰ متر,[205120],"[260634, 235045, 92336, 238909, 60239]","[69722, 33708, 38742, 18512, 41051]","[0.6568974256515503, 0.5845551490783691, 0.576...",اطلاعات کافی در متن‌های بازیابی‌شده نیست.
3,q4,semantic,آزمون مقیاس‌های تکاملی شیرخواران (Bayley) عملک...,کودکان ۱ تا ۴۲ ماهه,[300712],"[300712, 300712, 300712, 156840, 141053]","[39988, 39986, 39987, 42810, 51312]","[0.7965239882469177, 0.6681966185569763, 0.617...",آزمون مقیاس‌های تکاملی شیرخواران (Bayley) عملک...
4,q4,lexical,آزمون مقیاس‌های تکاملی شیرخواران (Bayley) عملک...,کودکان ۱ تا ۴۲ ماهه,[300712],"[335724, 100882, 265913, 79315, 213752]","[32440, 39293, 70957, 46426, 40624]","[0.6315245628356934, 0.6089589595794678, 0.600...",اطلاعات کافی در متن‌های بازیابی‌شده نیست.


In [26]:
out_path = "results_df.csv"
results_df.to_csv(out_path, index=False)
print("saved:", out_path)

saved: results_df.csv


In [24]:
import pandas as pd

if "is_acceptable" not in results_df.columns:
    results_df["is_acceptable"] = None

def manual_label(df: pd.DataFrame):
    df = df.copy()
    for idx, row in df.iterrows():
        print("=" * 100)
        print("id:", row["id"], "| mode:", row["embedding_model"])
        print("Q:", row["question"])
        print("Ref:", row["reference_answer"])
        print("\nAnswer:\n", str(row["answer"])[:1500])

        lab = input("\nAcceptable? (y/n/skip) ").strip().lower()
        if lab == "y":
            df.at[idx, "is_acceptable"] = True
        elif lab == "n":
            df.at[idx, "is_acceptable"] = False
        else:
            df.at[idx, "is_acceptable"] = None
    return df

results_df = manual_label(results_df)

acc_rate = results_df.dropna(subset=["is_acceptable"]).groupby("embedding_model")["is_acceptable"].mean() * 100
print("\nAcceptable rate (%) by embedding_model:\n", acc_rate)

overall = results_df.dropna(subset=["is_acceptable"])["is_acceptable"].mean() * 100
print("\nOverall acceptable rate (%):", overall)


id: q1 | mode: semantic
Q: آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند متر اعلام شد؟
Ref: بین ۳۰ تا ۴۰ متر

Answer:
 اطلاعات کافی در متن‌های بازیابی‌شده نیست.



Acceptable? (y/n/skip)  n


id: q1 | mode: lexical
Q: آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند متر اعلام شد؟
Ref: بین ۳۰ تا ۴۰ متر

Answer:
 اطلاعات کافی در متن‌های بازیابی‌شده نیست.



Acceptable? (y/n/skip)  n


id: q1 | mode: hybrid
Q: آبخور اسکله‌های «بندر بزرگ خلیج فارس» بین چند متر اعلام شد؟
Ref: بین ۳۰ تا ۴۰ متر

Answer:
 اطلاعات کافی در متن‌های بازیابی‌شده نیست.


...

id: q29 | mode: lexical
Q: در جلسه شورای فرهنگ عمومی بناب، از عدم پرداخت چه میزان از بودجه شهرداری به کتابخانه‌های عمومی انتقاد شد؟
Ref: نیم درصد بودجه شهرداری

Answer:
 مبلغ نیم درصد بودجه شهرداری به کتابخانه‌های عمومی انتقاد شد. این انتقاد از نایب رئیس شورای فرهنگ عمومی شهرستان بناب در جلسه شورای فرهنگ عمومی شهرستان بناب گفت شد.



Acceptable? (y/n/skip)  y


id: q29 | mode: hybrid
Q: در جلسه شورای فرهنگ عمومی بناب، از عدم پرداخت چه میزان از بودجه شهرداری به کتابخانه‌های عمومی انتقاد شد؟
Ref: نیم درصد بودجه شهرداری

Answer:
 از نیم درصد بودجه شهرداری به کتابخانه‌های عمومی انتقاد شد.



Acceptable? (y/n/skip)  y



Acceptable rate (%) by embedding_model:
 embedding_model
hybrid      53.333333
lexical     26.666667
semantic    53.333333
Name: is_acceptable, dtype: object

Overall acceptable rate (%): 44.44444444444444


In [ ]:
out_path = "results_df.csv"
results_df.to_csv(out_path, index=False)
print("saved:", out_path)

In [29]:
def pick_one_row_per_question(df, prefer_mode="hybrid"):
    mode_priority = [prefer_mode, "semantic", "lexical"]
    df = df.copy()
    df["mode_rank"] = df["embedding_model"].apply(lambda m: mode_priority.index(m) if m in mode_priority else 99)
    return df.sort_values(["id", "mode_rank"]).groupby("id", as_index=False).first().drop(columns=["mode_rank"])

good_ids = results_df.groupby("id")["is_acceptable"].apply(lambda s: (s == True).any())
good_ids = good_ids[good_ids].index.tolist()

bad_ids = results_df.groupby("id")["is_acceptable"].apply(lambda s: (s == True).any())
bad_ids = bad_ids[~bad_ids].index.tolist()

good_df = results_df[(results_df["id"].isin(good_ids)) & (results_df["is_acceptable"] == True)]
good_df = pick_one_row_per_question(good_df, prefer_mode="hybrid").head(3)

bad_df = results_df[(results_df["id"].isin(bad_ids)) & (results_df["is_acceptable"] == False)]
bad_df = pick_one_row_per_question(bad_df, prefer_mode="hybrid").head(3)

def show_examples_simple(df_ex, title):
    print("\n" + "#" * 50)
    print(title)
    print("#" * 50)

    for _, r in df_ex.iterrows():
        print("\n" + "-" * 100)
        print("id:", r["id"], "| mode:", r["embedding_model"])
        print("Q:", r["question"])
        print("Ref:", r["reference_answer"])
        print("Retrieved doc_ids:", r["retrieved_doc_ids"][:5] if isinstance(r["retrieved_doc_ids"], list) else r["retrieved_doc_ids"])
        print("\nAnswer:\n", str(r["answer"])[:1500])

show_examples_simple(good_df, "GOOD QUESTIONS (3 distinct questions)")
show_examples_simple(bad_df,  "BAD  QUESTIONS (3 distinct questions)")


##################################################
GOOD QUESTIONS (3 distinct questions)
##################################################

----------------------------------------------------------------------------------------------------
id: q11 | mode: hybrid
Q: طبق گفته مدیرکل سیاسی استانداری سمنان، انتخابات ۱۴۰۰ در سمنان به چه شیوه‌ای برگزار می‌شود؟
Ref: به‌طور دستی
Retrieved doc_ids: [73703, 67128, 82374, 67128, 27592]

Answer:
 مدیرکل سیاسی استانداری سمنان، سیدعباس دانایی، اعلام کرده است که انتخابات ۱۴۰۰ در سمنان به‌صورت دستی برگزار می‌شود.

----------------------------------------------------------------------------------------------------
id: q15 | mode: semantic
Q: احمد جباری تشکیل چه نوع دولتی را نیاز امروز کشور دانست؟
Ref: دولتی کارآمد، جهادی، میدانی و فسادستیز
Retrieved doc_ids: [103988, 88644, 88740, 71542, 72002]

Answer:
 احمد جباری از دولتی که به جوانان انقلابی متخصص و متوجه باشد نیاز دارد را در امروز دانست.

---------------------------------------------------------

<div dir="rtl" style="text-align: right;">

<h3>چرا خوب هستند؟</h3>

<h4>1) <span dir="ltr">q11</span> با بازیابی <span dir="ltr">hybrid</span></h4>
<ul>
  <li><b>پاسخ دقیق و هم‌راستا با مرجع است</li>
  <li><b>پاسخ از متن بازیابی‌شده قابل استخراج است</li>
</ul>


<h4>3) <span dir="ltr">q18</span> با بازیابی <span dir="ltr">lexical</span></h4>
<ul>
  <li><b>سؤال عددی و دقیق است:</b> سؤال مقدار مشخص می‌خواهد و پاسخ هم عدد دقیق <b>۱۰۵ قطعه</b> را می‌دهد.</li>
  <li><b>برای بازیابی لغوی مناسب است:</b> کلیدواژه‌هایی مثل اسفراین، فاز اول، قطعه زمین در متن  آمده‌اند </li>
</ul>

<hr/>

<h3> چرا این <span dir="ltr">BAD</span>‌ها بد هستند؟</h3>

<h4>1) <span dir="ltr">q1</span> با بازیابی <span dir="ltr">hybrid</span></h4>
<ul>
  <li><b>سؤال پاسخ دقیق دارد</b> (بین <b>۳۰ تا ۴۰ متر</b>) اما مدل می‌گوید «اطلاعات کافی نیست».</li>
  <li><b>مشکل اصلی: بازیابی (Retrieval)</b>
</ul>

<h4>2) <span dir="ltr">q16</span> با بازیابی <span dir="ltr">hybrid</span></h4>
<ul>
  <li><b>پاسخ مرجع عدد/ظرفیت مشخص است</b> («<b>۲۵ هزار TEU کانتینر</b>») اما مدل باز هم «اطلاعات کافی نیست» می‌گوید.</li>
  <li><b>این هم یک خطای بازیابی است

<h4>3) <span dir="ltr">q19</span> با بازیابی <span dir="ltr">hybrid</span></h4>
<ul>
  <li><b>سؤال:</b>فقط شهر قم یا کل استان؟: «<b>همه استان را شامل می‌شود</b>».</li>
  <li><b>اما مدل می‌گوید اطلاعات کافی نیست</b> — یعنی متنِ مرتبط (جمله‌ای که دامنه تعطیلی را مشخص می‌کند) در چانک‌های بازیابی‌شده موجود نبوده.</li>
  <li><b>نتیجه:</b> این هم نشان می‌دهد بازیابی نتوانسته همان جملهٔ کلیدیِ «دامنه تعطیلی» را وارد کانتکست کند.</li>
</ul>

<hr/>


<div dir="rtl" style="text-align: right;">


در ارزیابی دستی انجام‌ شده روی مجموعه‌ی سوال‌ها نرخ پاسخ‌ های «قابل قبول» به‌صورت کلی **۴۴.۴٪** بود. وقتی نتایج را به تفکیک روش بازیابی بررسی کردم عملکرد دو روش **معنایی** و **ترکیبی** بهتر از روش لغوی بود:  
- <span dir="ltr">semantic</span>: **۵۳.۳٪**  
- <span dir="ltr">hybrid</span>: **۵۳.۳٪**  
- <span dir="ltr">lexical</span>: **۲۶.۷٪**  

این اختلاف نشان می‌ دهد که برای سوال‌ های فارسی در این دیتاست بازیابی مبتنی بر معنا (و همچنین ترکیب آن با روش لغوی) معمولا شانس بیشتری برای رسیدن به چانک مرتبط دارد در حالی که روش صرفا لغوی در بسیاری از موارد نتوانسته سند درست را در نتایج بالا بیاورد.

بزرگ‌ترین محدودیت در نمونه‌ های ناموفق، **ضعف در مرحله‌ی بازیابی** بود. در چند سوال که پاسخ مرجع یک عدد/بازه/ظرفیت مشخص داشت، اگر چانک وارد <span dir="ltr">top-k</span> نمی‌ شد، مدل طبق پرامپت از حدس‌ زدن خودداری می‌ کرد و نتیجه به شکل «اطلاعات کافی در متن‌های بازیابی‌شده نیست» نشان داده میشد.
برای نسخه‌ی بعدی سیستم، چند بهبود مشخص به نظرم می‌ رسد. اول، **بهبود چانک‌ کردن**: در این پروژه چانک‌ها بر اساس تعداد کاراکتر ساخته شدند و ممکن است جمله‌ کلیدی یا عدد مهم دقیقا در مرز چانک قرار بگیرد چانک‌ بندی مبتنی بر جمله/پاراگراف به‌ همراه <span dir="ltr">overlap</span> مناسب می‌تواند این مشکل را کمتر کند. دوم، **تقویت بازیابی**: افزایش مقدار <span dir="ltr">top_k</span> (مثلاً ۵ به ۱۰) برای سوال‌های دقیق و همچنین استفاده از روش‌های ترکیبی قوی‌تر (مثل ترکیب رتبه‌بندی معنایی و لغوی به‌جای صرفاً چسباندن بردارها) احتمال رسیدن به سند درست را بالا می‌ برد. سوم، **مدل‌های بهتر**: هم در بخش embedding و هم در مدل زبانی، استفاده از مدل‌های قوی‌ تر یا تنظیم دقیق‌ تر پارامترهای تولید می‌تواند باعث شود پاسخ‌ ها کمتر کلی باشند و بیشتر به شواهد متن تکیه کنند. در کنار این‌ ها نرمال‌ سازی بهتر متن فارسی (یکسان‌سازی حروف، حذف نویزهای متداول و یکدست‌کردن علائم) هم می‌تواند روی کیفیت بازیابی و در نتیجه روی نرخ پاسخ قابل‌ قبول اثر مثبت بگذارد.

</div>
